In [49]:
#! jupyter nbconvert --to script get_maxi_products_prices.ipynb

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import date
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
#import schedule
import time
import gc
from tqdm import tqdm

from selenium.webdriver.chrome.options import Options
chrome_options = Options()  
#chrome_options.add_argument("--headless")
chrome_options.add_experimental_option('prefs', {'intl.accept_languages': 'en,en_US'})

In [8]:
def get_product(categorie_url, name):
    
    folder = str(date.today())
    today = date.today()
    store = 'maxi'
    
    if not os.path.exists(folder):
        os.makedirs(folder)
        print("Directory " , folder ,  " Created ")
    else:    
        print("Directory " , folder ,  " already exists") 
        
    if not os.path.exists(folder+'/'+store):
        os.makedirs(folder+'/'+store)
        print("Directory " , store ,  " Created ")
    else:    
        print("Directory " , store ,  " already exists") 
        
    driver = webdriver.Chrome("/Applications/chromedriver")#options=chrome_options)
    driver.get(categorie_url)
    time.sleep(7)
    print(driver.find_element_by_xpath('/html/body/div[2]/div/div[1]/div[2]/div/span[3]/span/button').text)
    if driver.find_element_by_xpath('/html/body/div[2]/div/div[1]/div[2]/div/span[3]/span/button').text == 'EN':
        driver.find_element_by_xpath('/html/body/div[2]/div/div[1]/div[2]/div/span[3]/span/button').click()
    time.sleep(7)
    while True:
        try:
            element = driver.find_element_by_css_selector('.primary-button')
            coordinates = element.location_once_scrolled_into_view
            driver.execute_script('window.scrollTo({}, {});'.format(coordinates['x'], coordinates['y']))
            driver.find_element_by_css_selector('.primary-button').click()
            time.sleep(7)
        except Exception as e:
            print(e)
            break
    time.sleep(7)
    
    print("Web Page Loaded")
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    items = soup.find_all('li', {'class': 'product-tile-group__item'})
    attributs = []
    ratios = []
    sales = []
    for item in items:
        attribut = item.find('div', {'class': 'product-tracking'}).attrs
        attributs.append(attribut['data-track-products-array'])
        ratio = item.find('span', {'class' : 'price comparison-price-list__item__price'})
        try:
            a = ratio.text
        except Exception as e:
            print(e)
            a = np.nan
        ratios.append(a)
        sale = item.find('div', {'class' : 'product-badge__text product-badge__text--product-tile'})
        try:
            b = sale.text
        except Exception as e:
            b = np.nan
        sales.append(b) 
        
    products = pd.DataFrame(attributs, columns={'attributs'})

    products['ratios'] = ratios
    products['sales'] = sales
    products['name'] = name
    products['date'] = str(today)
    
    filename=name+'_'+str(today)+'.csv'
    
    products.to_csv(folder+'/'+store+'/'+filename,index=False)
    print(products.shape)
    print(filename,"is Saved")
    
    driver.quit()
    time.sleep(10)
    
    gc.collect()
    return products

In [9]:
def run_app():
    bakery = 'https://www.maxi.ca/Food/Bakery/c/MAX001003000000'
    beauty = 'https://www.maxi.ca/Home-%26-Lifestyle/Beauty/c/MAX002004000000'
    dairy = 'https://www.maxi.ca/Food/Dairy-and-Eggs/c/MAX001005000000'
    deli = 'https://www.maxi.ca/Food/Deli-%26-Ready-Meals/c/MAX001002000000'
    drinks = 'https://www.maxi.ca/Food/Drinks/c/MAX001006000000'
    frozen  = 'https://www.maxi.ca/Food/Frozen/c/MAX001007000000'
    vegetables = 'https://www.maxi.ca/Food/Fruits-%26-Vegetables/c/MAX001001000000'
    household = 'https://www.maxi.ca/Home-%26-Lifestyle/Household-Supplies/c/MAX002003000000'
    meat = 'https://www.maxi.ca/Food/Meat-%26-Seafood/c/MAX001004000000'
    pantry = 'https://www.maxi.ca/Food/Pantry/c/MAX001008000000'
    personal = 'https://www.maxi.ca/Personal-Care/c/MAX002005000000'

    a =[bakery,dairy,deli,frozen,vegetables,meat,pantry]
    b = ['bakery','dairy','deli','frozen','vegetables','meat','pantry']

    for i, j in tqdm(zip(a,b)):
            products = get_product(i, j)
            while True:
                if products.shape[0] == 0:
                    products = get_product(i, j)
                else:
                    print(j,' is Done.')
                    break
        
run_app()

## OR  
# schedule.every().day.at("10:30").do(run_app)

# while True:
#     schedule.run_pending()
#     time.sleep(1)

0it [00:00, ?it/s]

Directory  2020-10-13  already exists
Directory  maxi  already exists


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=86.0.4240.75)
